# 4. Modelo de Produção - Dataset 2

Este notebook documenta o desenvolvimento do modelo de Análise de Sentimento da API.
O objetivo é sair de uma solução simples (Baseline) e refinar o modelo tratando problemas reais como o desbalanceamento de classes, até chegar na melhor solução para produção.

## Roteiro de Experimentos
- **Experimento 1 (Baseline)**: Modelo simples sem tratamento de desbalanceamento.
- **Experimento 2 (Desbalanceamento)**: Comparação entre Class Weights e SMOTE.
- **Experimento 3 (Fine Tuning)**: Otimização de hiperparâmetros com GridSearchCV.
- **Final**: Seleção e exportação do modelo vencedor.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import joblib
import os
from imblearn.over_sampling import SMOTE

sns.set_style("whitegrid")

## Carregamento e Preparação dos Dados

In [ ]:
# Carregar dataset pré-processado
file_path = '../datasets/processed/reviews_dataset2_advanced.csv'
df = pd.read_csv(file_path)
df = df.dropna(subset=['processed_text'])

X = df['processed_text']
y = df['sentiment']

# Divisão Estratificada
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Treino: {X_train.shape[0]} amostras | Teste: {X_test.shape[0]} amostras")

### Verificando Desbalanceamento

In [ ]:
dist = y_train.value_counts(normalize=True) * 100
print("Distribuição das Classes no Treino:")
print(dist)

## Baseline (Sem Tratamento)
Vamos treinar um modelo simples de Regressão Logística ignorando o desbalanceamento. Isso serve como linha de base para sabermos se nossas otimizações futuras realmente ajudam.

In [ ]:
# Vetorização simples para o baseline
vec_base = TfidfVectorizer(max_features=5000)
X_train_base = vec_base.fit_transform(X_train)
X_test_base = vec_base.transform(X_test)

# Modelo Padrão
model_base = LogisticRegression(random_state=1337, max_iter=1000)
model_base.fit(X_train_base, y_train)

y_pred_base = model_base.predict(X_test_base)

print("--- Resultado Baseline ---")
print(classification_report(y_test, y_pred_base))

**Análise (Baseline):**
Observe o **Recall** da classe minoritária (0). O valor está baixo, o que significa que o modelo está apenas "chutando" a classe majoritária para maximizar a Acurácia, ignorando as críticas negativas.

## Tratando o Desbalanceamento
Testaremos duas técnicas populares para resolver o problema identificado no baseline:

1.  **Class Weights**: Penalizar o modelo mais fortemente quando ele erra a classe minoritária.
2.  **SMOTE**: Gerar exemplos sintéticos da classe minoritária para equilibrar o treino.

In [ ]:
# --- Abordagem A: Class Weights ---
print("Treinando com Class Weights...")
model_weighted = LogisticRegression(class_weight='balanced', random_state=1337, max_iter=1000)
model_weighted.fit(X_train_base, y_train)
y_pred_weighted = model_weighted.predict(X_test_base)

# --- Abordagem B: SMOTE ---
print("Aplicando SMOTE...")
smote = SMOTE(random_state=1337)
X_train_res, y_train_res = smote.fit_resample(X_train_base, y_train)

print(f"Novo shape de treino (SMOTE): {X_train_res.shape}")

model_smote = LogisticRegression(random_state=1337, max_iter=1000)
model_smote.fit(X_train_res, y_train_res)
y_pred_smote = model_smote.predict(X_test_base)

# Comparação
print("\n--- Class Weights Balanced ---")
print(classification_report(y_test, y_pred_weighted))

print("\n--- SMOTE ---")
print(classification_report(y_test, y_pred_smote))

**Decisão:**
Geralmente, em NLP com alta dimensionalidade (TF-IDF), o **Class Weight** tende a funcionar tão bem quanto o SMOTE, com a vantagem de ser computacionalmente mais leve e não introduzir ruído sintético. Vamos seguir otimizando a versão com *Class Weights*.

## Fine Tuning (GridSearch)
Agora que decidimos usar `class_weight='balanced'`, vamos encontrar o melhor algoritmo e parâmetros.
1.  **Logistic Regression** (Probabilístico)
2.  **Linear SVC** (Margem máxima, geralmente ótimo para texto)


In [ ]:
# Pipelines com Class Weight Balanced
pipe_lr = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(class_weight='balanced', max_iter=1000, random_state=1337))
])

pipe_svc = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LinearSVC(class_weight='balanced', random_state=1337))
])

pipe_lr_final = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2), sublinear_tf=False)), # Fixamos o que já venceu
    ('clf', LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42, solver='liblinear'))
])

# GRID DE PARÂMETROS

# Resultados de GridSearch anteriores:
#{'clf__C': 10, 'tfidf__max_features': 10000, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 2), 'tfidf__sublinear_tf': False}
#{'clf__C': 10, 'tfidf__max_features': 15000, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 2), 'tfidf__sublinear_tf': False}
#{'clf__C': 10, 'clf__penalty': 'l2', 'tfidf__max_features': 15000, 'tfidf__min_df': 5}

#param_grid = {
#    'tfidf__ngram_range': [(1, 1), (1, 2)],
#    'tfidf__max_features': [5000, 10000],
#    'tfidf__min_df': [2, 5],            # Palavra deve aparecer em pelo menos 2 ou 5 docs
#    'tfidf__sublinear_tf': [True, False], # Escala logarítmica para frequências
#    'clf__C': [0.1, 1, 10]
#}

# Expandindo os parâmetros em torno dos valores que o GridSearch retornou como melhores.
param_grid = {
    'tfidf__ngram_range': [(1, 2)], # Já sabemos que (1,2) é melhor que (1,1)
    'tfidf__max_features': [10000, 15000], 
    'tfidf__min_df': [5, 10],       # Testando se limpar mais ajuda a generalizar
    'tfidf__sublinear_tf': [False], 
    'clf__C': [10, 20, 50, 100]    # Expandindo o C para ver até onde vai
}

# Grade para comparar as penalidades
param_grid_l1_l2 = {
    'tfidf__max_features': [15000, 20000], # Testando se 20k ajuda ainda mais
    'tfidf__min_df': [5],
    'clf__C': [5, 10, 15],                 # Testando valores próximos ao 10
    'clf__penalty': ['l1', 'l2']           # L1 vs L2
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1337)

print("Rodando GridSearch para Logistic Regression...")
grid_lr = GridSearchCV(pipe_lr_final, param_grid_l1_l2, cv=cv, scoring='f1', n_jobs=-1)
grid_lr.fit(X_train, y_train)
print(f"Melhor F1 (LR): {grid_lr.best_score_:.4f}")
print(f"Melhores Parâmetros (LR): {grid_lr.best_params_}")

# Como o LR consistentemente retorna resultados melhores, deixaremos o SVC de lado.
# print("\nRodando GridSearch para Linear SVC...")
# grid_svc = GridSearchCV(pipe_svc, param_grid, cv=cv, scoring='f1', n_jobs=-1)
# grid_svc.fit(X_train, y_train)
# print(f"Melhor F1 (SVC): {grid_svc.best_score_:.4f}")
# print(f"Melhores Parâmetros (SVC): {grid_svc.best_params_}")

## Resultado Final
Selecionamos o modelo vencedor e avaliamos no conjunto de teste.

In [ ]:
# Instancia o modelo com os parâmetros finais
final_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        ngram_range=(1, 2), 
        max_features=15000, 
        min_df=5, 
        sublinear_tf=False
    )),
    ('clf', LogisticRegression(
        C=10, 
        penalty='l2', 
        solver='liblinear', # Garantindo compatibilidade
        class_weight='balanced', 
        random_state=1337
    ))
])

# Treino
final_pipeline.fit(X_train, y_train)

# Avaliação no conjunto de teste
y_pred = final_pipeline.predict(X_test)

print("--- RELATÓRIO DE DESEMPENHO FINAL ---")
print(classification_report(y_test, y_pred))

# Matriz de Confusão para análise de erros
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens')
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de Confusão Final')
plt.show()

### Salvar Modelo de Produção

In [ ]:
model_path = '../models/production_model.pkl'
joblib.dump(final_pipeline, model_path)